In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv("BiasEst.csv")    
x = df["x"].to_numpy()
n = len(x)

# Estimator: sample maximum
Tn = np.max(x)

# Jackknife
jack_vals = np.empty(n, dtype=float)
for i in range(n):
    jack_vals[i] = np.max(np.delete(x, i))

T_dot = jack_vals.mean()
bias_jack = (n - 1) * (T_dot - Tn)

# Bootstrap
B = 10000
rng = np.random.default_rng(0)  # fixed seed for reproducibility

boot_vals = np.empty(B, dtype=float)
for b in range(B):
    sample = rng.choice(x, size=n, replace=True)
    boot_vals[b] = np.max(sample)

bias_boot = boot_vals.mean() - Tn

# Printout
print(f"n = {n}")
print(f"T_n = max(x) = {Tn:.6f}")
print(f"Jackknife bias estimate = {bias_jack:.6f}")
print(f"Bootstrap bias estimate (B={B}) = {bias_boot:.6f}")


n = 30
T_n = max(x) = 1.983800
Jackknife bias estimate = -0.091253
Bootstrap bias estimate (B=10000) = -0.039243


In [5]:
# With MC method

import numpy as np

K = 10000          # MC replications
n = 30
theta = 2.0
B = 500            
rng = np.random.default_rng(123)

theta_vec = np.empty(K)
bias_jack = np.empty(K)
bias_boot = np.empty(K)

# True bias of T_n = max(X_i) for Unif(0, theta)
true_bias = -theta / (n + 1)

# Fast jackknife bias for max

def jackknife_bias_max(x: np.ndarray) -> float:
    n = x.size
    xs = np.sort(x)
    m1 = xs[-1]      # max
    m2 = xs[-2]      # second max
    c = np.sum(x == m1) 

    # Average of leave-one-out maxima:
    # if max appears more than once: all leave-one-out maxima are still m1
    # if max appears once: one leave-one-out max becomes m2, others stay m1
    if c > 1:
        avg_leave1 = m1
    else:
        avg_leave1 = ((n - 1) * m1 + m2) / n

    # Jackknife bias estimate: (n-1)(T_. - T)
    return (n - 1) * (avg_leave1 - m1)

# Monte Carlo loop
for k in range(K):
    # simulate sample
    x = rng.uniform(0.0, theta, size=n)

    # T_n = max
    Tn = x.max()
    theta_vec[k] = Tn

    # jackknife bias estimate
    bias_jack[k] = jackknife_bias_max(x)

    # bootstrap bias estimate: E*(T*) - T
    idx = rng.integers(0, n, size=(B, n))
    boot_max = x[idx].max(axis=1)
    bias_boot[k] = boot_max.mean() - Tn

# Summaries
def summarize(arr):
    return {
        "mean": np.mean(arr),
        "sd": np.std(arr, ddof=1),
        "mse_vs_true_bias": np.mean((arr - true_bias) ** 2),
        "mae_vs_true_bias": np.mean(np.abs(arr - true_bias)),
    }

print(f"True bias = {true_bias:.6f}\n")

print("E[T_n] check:")
print(f"  MC mean of T_n = {theta_vec.mean():.6f}")
print(f"  True E[T_n]    = {(n/(n+1)*theta):.6f}\n")

sj = summarize(bias_jack)
sb = summarize(bias_boot)

print("Jackknife bias estimator:")
print(f"  mean = {sj['mean']:.6f}, sd = {sj['sd']:.6f}")
print(f"  MSE vs true bias = {sj['mse_vs_true_bias']:.6f}")
print(f"  MAE vs true bias = {sj['mae_vs_true_bias']:.6f}\n")

print("Bootstrap bias estimator:")
print(f"  mean = {sb['mean']:.6f}, sd = {sb['sd']:.6f}")
print(f"  MSE vs true bias = {sb['mse_vs_true_bias']:.6f}")
print(f"  MAE vs true bias = {sb['mae_vs_true_bias']:.6f}")


True bias = -0.064516

E[T_n] check:
  MC mean of T_n = 1.935274
  True E[T_n]    = 1.935484

Jackknife bias estimator:
  mean = -0.062752, sd = 0.060396
  MSE vs true bias = 0.003650
  MAE vs true bias = 0.045456

Bootstrap bias estimator:
  mean = -0.035800, sd = 0.024039
  MSE vs true bias = 0.001402
  MAE vs true bias = 0.033751
